In [9]:
import pandas as pd
import dbconfig as dbConf

In [44]:
def GetPatten():
    # MDM DB 연결
    zeroDb = dbConf.DbConfig("zero")
    zeroDb.opendb()
    #print("접속완료")

    sql = ("""
           select stan
                , 3cnt
                , 5cnt
                , 10cnt
                , 30cnt
             from in_list_2
            limit 15
           """
           )

    # 수집 대상 DB정보 및 테이블, 쿼리 조회
    source = zeroDb.select(sql) 
    zeroDb.closedb()

    return pd.DataFrame(source)

In [97]:
df = GetPatten()
stan  = list(range(0,1))
cnt3  = list(range(0,1))
cnt5  = list(range(0,1))
cnt10 = list(range(0,1))
cnt30 = list(range(0,1))

for df_stan, df_cnt3, df_cnt5, df_cnt10, df_cnt30 in zip(df['stan'], df['3cnt'], df['5cnt'], df['10cnt'], df['30cnt']):
    stan.append(int(df_stan))
    cnt3.append(int(df_cnt3))
    cnt5.append(int(df_cnt5))
    cnt10.append(int(df_cnt10))
    cnt30.append(int(df_cnt30))

print(stan[1])
print(cnt3[1])
print(cnt5[1])
print(cnt10[1])
print(cnt30[1])

974
2
1
1
2
